## Import Libraries

In [3]:
!pip install llama-index
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from llama_index import SimpleDirectoryReader, GPTVectorStoreIndex, LLMPredictor, ServiceContext, PromptHelper, load_index_from_storage, StorageContext
from langchain import OpenAI
import gradio as gr
import os

## Scrape Data From Website(s)

In [19]:
# Use this piece of code for testing new custom BeautifulSoup parsers

# html_doc = requests.get("https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/")
# soup = BeautifulSoup(html_doc.text, 'html.parser')
# heading = soup.find('h1').text.strip()
# print(heading)
# article = soup.find('article')
# if article:
#     article_text = ''
#     for div in article.find_all('div',{'class': 'tdb-block-inner td-fix-index'}):
#          for tag in div:
#              if tag.name in ['p']:
#                  article_text += tag.text.strip() + '\n'
#     print(article_text)
# else:
#     print('Article content not found.')

# #write title and text to the file
# os.mkdir("docs")
# file_name = '/content/docs/' + "url.txt"
# with open(file_name, 'w') as file:
#      file.write(article_text)
#----------------------------------------------------------OR----------------------------------------------------------
#------------------------------------------DEPENDING ON THE WEBSITE YOU CHOOSE-----------------------------------------
import requests
from bs4 import BeautifulSoup

# URL to scrape
url = "https://www.analyticsvidhya.com/blog/2016/09/40-interview-questions-asked-at-startups-in-machine-learning-data-science/"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content using Beautiful Soup
soup = BeautifulSoup(response.content, "html.parser")

# Find the relevant HTML elements that contain the questions and answers
content = soup.find_all("p")  # Adjust the element type if needed

# Extract the text from the elements
content_text = [answer.get_text(strip=True) for answer in content]

text_content = ''
os.mkdir("docs")
file_name = '/content/docs/' + "content.txt"
for answer in content_text:
    text_content += answer + '\n'
with open(file_name, 'w') as file:
     file.write(text_content)
print(text_content)
# Beautiful soup logic to be exported to langchain.document_loaders.webpage.py
# Example: transcript = soup.select_one("td[class='scrtext']").text
# BS4 documentation can be found here: https://www.crummy.com/software/BeautifulSoup/bs4/doc/




Careful! These questions can make you think THRICE!
Machine learning and data science are being looked as the drivers of the next industrial revolution happening in the world today. This also means that there are numerousexciting startupslooking for data scientists. What could be a better start for your aspiring career!
However, still, getting into these roles is not easy. You obviously need to get excited about the idea, team and the vision of the company. You might also find some real difficult techincal questions on your way. The set of questions asked depend on what does the startup do. Do they provide consulting? Do they build ML products ? You should always find this out prior to beginning your interview preparation.
To help you prepare for your next interview, I’ve prepared a list of 40 plausible & tricky questions which are likely to come across your way in interviews. If you can answer and understand these question, rest assured, you will give a tough fight in your job inte

## Add OpenAI Chatgpt API key

In [20]:
os.environ["OPENAI_API_KEY"] = 'sk-ey7ZW3svyUh1YWhFYnNpT3BlbkFJ8SQomSiZBofbiKuErE'
# Don't abuse the api key

In [21]:


# def construct_index(directory_path):
#     num_outputs = 512

#     llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.7, model_name="text-davinci-003", max_tokens=num_outputs))

#     service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

#     docs = SimpleDirectoryReader(directory_path).load_data()

#     index = GPTVectorStoreIndex.from_documents(docs, service_context=service_context)

#     index.storage_context.persist('index.json')

#     return index

# def chatbot(input_text):
#     index = GPTVectorStoreIndex.load_from_disk('index.json')
#     response = index.query(input_text, response_mode="compact")
#     return response.response

# iface = gr.Interface(fn=chatbot,
#                      inputs=gr.inputs.Textbox(lines=7, label="Enter your text"),
#                      outputs="text",
#                      title="Custom-trained AI Chatbot")

# index = construct_index("docs")
# iface.launch(share=True)

## Initialize the Model

In [22]:
# Adapted based on LlamaIndex documentation https://gpt-index.readthedocs.io/en/latest/index.html
# and Dan Shipper's work https://www.lennysnewsletter.com/p/i-built-a-lenny-chatbot-using-gpt
from llama_index import StorageContext, load_index_from_storage
from IPython.display import Markdown, display

def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens
    num_outputs = 256
    # set maximum chunk overlap
    max_chunk_overlap = 20
    # set chunk size limit
    chunk_size_limit = 600

    # define LLM (ChatGPT gpt-3.5-turbo)
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="gpt-3.5-turbo", max_tokens=num_outputs))
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)
 
    documents = SimpleDirectoryReader(directory_path).load_data()
    
    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

    index = GPTVectorStoreIndex.from_documents(
        documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper, service_context=service_context
    )

    index.storage_context.persist(persist_dir="/content/index")

    return index


def ask_me_anything(question):
    storagee = StorageContext.from_defaults(persist_dir="/content/index")
    index = load_index_from_storage(storagee)
    # response = index.query(question, response_mode="compact")
    
    query_engine = index.as_query_engine()
    response = query_engine.query(question)

    return response.response
    # display(Markdown(f"You asked: <b>{question}</b>"))
    # display(Markdown(f"Bot says: <b>{response.response}</b>"))

In [23]:
construct_index('/content/docs')

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:696: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [24]:
question = "How is kNN different from kmeans clustering?"

In [25]:
# Run Reddit chatbot
ask_me_anything(question)

'\nAnswer:Don’t get mislead by ‘k’ in their names. You should know that the fundamental difference between both these algorithms is, kmeans is unsupervised in nature and kNN is supervised in nature. kmeans is a clustering algorithm. kNN is a classification (or regression) algorithm. kmeans algorithm partitions a data set into clusters such that a cluster formed is homogeneous and the points in each cluster are close to each other. The algorithm tries to maintain enough separability between these clusters. Due to unsupervised nature, the clusters have no labels. kNN algorithm tries to classify an unlabeled observation based on its k (can be any number ) surrounding neighbors. It is also known as lazy learner because it involves minimal training of model. Hence, it doesn’t use training data to make generalization on unseen data set.'

In [26]:
iface = gr.Interface(fn=ask_me_anything,
                     inputs=gr.inputs.Textbox(lines=7, label="Enter your text"),
                     outputs="text",
                     title="Custom-trained AI Chatbot")


/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


In [27]:
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5afab4fb0e796fbdcf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
